<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl_2/blob/main/ML_T5_traductor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Descargamos de "manythings.org" el dataset spa-eng.zip. Con pares de palabras en inglés-español.

In [1]:
!wget http://www.manythings.org/anki/spa-eng.zip
!unzip spa-eng.zip
!ls -l

--2022-12-03 00:22:43--  http://www.manythings.org/anki/spa-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5336731 (5.1M) [application/zip]
Saving to: ‘spa-eng.zip.1’

spa-eng.zip.1       100%[===================>]   5.09M  2.82MB/s    in 1.8s    

2022-12-03 00:22:45 (2.82 MB/s) - ‘spa-eng.zip.1’ saved [5336731/5336731]

Archive:  spa-eng.zip
replace _about.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
replace spa.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: no
total 30752
-rw-r--r-- 1 root root     1441 Sep  6 03:10 _about.txt
-rw-r--r-- 1 root root    55069 Dec  3 00:09 accuracy_model.png
-rw-r--r-- 1 root root    39767 Dec  3 00:09 decoder_model.png
drwx------ 5 root root     4096 Dec  3 00:05 drive
-rw-r--r-- 1 root root    26161 Dec  3 00:06 encoder_model.png
drwxr-xr-x 1 root root     4096 Dec  1 20:08 sample_dat

In [2]:
import re
SENTENCE_COUNT = 1000# Para el LSTM, que sean poquitas
input_sentences = []
input_translated_sentences = []
output_translated_sentences = []
sentences_file_path = '/content/spa.txt'#Cargamos el dataset
count = 0
for line in open(sentences_file_path, encoding='utf-8'):
    if count < SENTENCE_COUNT:
        if '\t' in line:
            line = re.sub(r'[^\w\s]','', line)# Primer valor: English,  
            line_values = line.rstrip().split('\t')
            input_sentences.append(line_values[0])

            #Tokens de incio y final
            input_translated_sentence = '走 ' + line_values[1]#Segundo: Frase en español
            output_translated_sentence = line_values[1] + ' 停'

            input_translated_sentences.append(input_translated_sentence)
            output_translated_sentences.append(output_translated_sentence)
            count += 1
    else:
        break

In [3]:
from keras.preprocessing.text import Tokenizer
VOCABULARY_SIZE = 10000 #máximo vocalubario posible 
# Tokenizamos
input_tokenizer = Tokenizer(num_words=VOCABULARY_SIZE)
input_tokenizer.fit_on_texts(input_sentences)
# Convertimos a secuencias 
input_sequences = input_tokenizer.texts_to_sequences(input_sentences)
#Diccionario de palabras del input
word_index_dict = input_tokenizer.word_index
max_input_length = max(len(seq) for seq in input_sequences)

In [4]:
# Todo el proceso anterior pero para el output.
output_tokenizer = Tokenizer(num_words=VOCABULARY_SIZE)
output_tokenizer.fit_on_texts(input_translated_sentences + output_translated_sentences)
input_translated_sequences = output_tokenizer.texts_to_sequences(input_translated_sentences)
output_translated_sequences = output_tokenizer.texts_to_sequences(output_translated_sentences)
translated_word_index_dict = output_tokenizer.word_index
max_output_length = max([len(seq) for seq in output_translated_sequences])

# Padding

In [5]:
from keras.utils import pad_sequences

encoder_input_sequences = pad_sequences(input_sequences, maxlen=max_input_length)
decoder_input_sequences = pad_sequences(input_translated_sequences, maxlen=max_output_length, padding='post')
decoder_output_sequences = pad_sequences(output_translated_sequences, maxlen=max_output_length, padding='post')

Cargamos en Drive el **glove.6B.300d**, el embedding del diccionario en inglés.

Y creamos el embedding word 2 vec.

In [6]:
import numpy as np
from numpy import array, asarray, zeros
embeddings_file_path = '/content/drive/MyDrive/glove.6B.300d.txt'
embeddings_dict_w2v = dict()
w2v_file = open(embeddings_file_path)

for line in w2v_file:
    records = line.split() # Turn into array with word on first position and embeddings as rest of line.
    word = records[0]
    vector_dim = asarray(records[1:], dtype='float32') # Take rest of embeddings out.
    embeddings_dict_w2v[word] = vector_dim # Add to embe

In [8]:
EMBEDDING_SIZE = 300 #tamaño de los embeddings
word_count = VOCABULARY_SIZE
embeddings_matrix_w2v = zeros((word_count, EMBEDDING_SIZE)) 
for word, index in word_index_dict.items():
    # Attempts to get word embeddings for specific word.
    embeddings_vector = embeddings_dict_w2v.get(word)
    if embeddings_vector is not None:
        embeddings_matrix_w2v[index] = embeddings_vector

In [9]:
from keras.utils import to_categorical
twc = len(translated_word_index_dict) + 1
decoder_targets_OHE = to_categorical(decoder_output_sequences, num_classes=twc)

In [10]:
from keras.layers import Input, LSTM, GRU, Dense, Embedding, Dropout, GlobalMaxPooling1D 
from keras.models import Model
from IPython.display import Image
from keras.utils import plot_model

# Capa del input.
encoder_input = Input(shape=(max_input_length,), name='EncoderInput')

# Se conecta el input con la capa de embeddings
encoder_input_x = Embedding(word_count,
                          EMBEDDING_SIZE,
                          weights=[embeddings_matrix_w2v],
                          input_length=max_input_length,
                          name='EncoderEmbeddingLayer')(encoder_input)

# El output del encoder y los estados h/c se extraen para usarse en el decoder.
encoder_output, h, c = LSTM(1024, 
                            return_state=True,
                            name='EncoderLSTM')(encoder_input_x)
encoder_states = [h, c]

Creamos el modelo **encoder**

In [11]:
encoder_model = Model(encoder_input, encoder_states)

Se repite el proceso para definir el **decoder**

**SBW-vectors-300-min5.txt** es el embbeding en español.

In [12]:
import numpy as np
from numpy import array, asarray, zeros
embeddings_file_path = '/content/drive/MyDrive/SBW-vectors-300-min5.txt'
embeddings_dict = dict()
glove_file = open(embeddings_file_path)

for line in glove_file:
    records = line.split() # Turn into array with word on first position and embeddings as rest of line.
    word = records[0]
    vector_dim = asarray(records[1:], dtype='float32') # Take rest of embeddings out.
    embeddings_dict[word] = vector_dim # Add to embeddings_dict as word:embeddings.

In [13]:
EMBEDDING_SIZE = 300
word_count = min(VOCABULARY_SIZE, twc)
embeddings_matrix = zeros((twc, EMBEDDING_SIZE)) 
for word, index in translated_word_index_dict.items():
    # Attempts to get word embeddings for specific word.
    embeddings_vector = embeddings_dict.get(word)
    if embeddings_vector is not None:
        embeddings_matrix[index] = embeddings_vector

Definición del encoder:

In [15]:
# La capa de input/embeddings, ahora se carga la capa de embeddings inicializada con GloVe.
decoder_input = Input(shape=(max_output_length,), name='DecoderInput')
decoder_embedding = Embedding(twc, EMBEDDING_SIZE, 
                            weights=[embeddings_matrix],
                          input_length=twc, name='DecoderEmbeddingLayer')

# La capa de input se conecta con la de embeddings 
decoder_input_x = decoder_embedding(decoder_input)

# El decoder LSTM se crea y se conecta con la capa de decoder_input 
decoder_LSTM = LSTM(1024, return_state=True, return_sequences=True, name='DecoderLSTM')
decoder_output, _, _ = decoder_LSTM(decoder_input_x, initial_state=encoder_states)

# Se pasa el output por dos capas densas para poder hacer las predicciones. 
decoder_dense = Dense(1024, activation='relu', name='DenseLayer') 
decoder_dense_softmax = Dense(twc, activation='softmax', name='DenseSoftmaxLayer')

decoder_output = decoder_dense(decoder_output)
decoder_output = decoder_dense_softmax(decoder_output)

In [16]:
# Se generan las capas de h/c.
decoder_state_input_h = Input(shape=(1024,), name='InputHStep')
decoder_state_input_c = Input(shape=(1024,), name='InputCStep')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

#el decoder debe aceptar una sola palabra para la capa de input al recorrer toda la oración.
decoder_input_step = Input(shape=(1,), name='DecoderInputStep')
decoder_input_x_step = decoder_embedding(decoder_input_step)
decoder_output_step, h, c = decoder_LSTM(decoder_input_x_step, initial_state=decoder_states_inputs)

# Estados finales en un array 
decoder_states = [h, c]


#El output del decor se pasa por dos capas densas
decoder_output_step = decoder_dense(decoder_output_step)
decoder_output_step = decoder_dense_softmax(decoder_output_step)

# Modelo decoder
decoder_model = Model(
    [decoder_input_step] + decoder_states_inputs, # initial input step AND initial h/c layers 
    [decoder_output_step] + decoder_states # output input step AND output h/c from decoder LSTM 
)

Hasta ahora ya se tiene el encoder-decoder. Después se define el modelo de precisión, recibe encoder, decoder y obtenemos un decoder de salida.

In [23]:
from keras.models import Model
from keras.optimizers import RMSprop
accuracy_model = Model([encoder_input, decoder_input], decoder_output)
accuracy_model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

Lo entrenamos con 10 epocas y batch de 20, también le damos $10\%$ de datos para validación.

In [24]:
r = accuracy_model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_OHE,
    batch_size=20,
    epochs=20,
    validation_split=0.1,
)

Epoch 1/20
45/45 [==============================] - 5s 43ms/step - loss: 0.1680 - accuracy: 0.9190 - val_loss: 5.7855 - val_accuracy: 0.6929
Epoch 2/20
45/45 [==============================] - 1s 21ms/step - loss: 0.1488 - accuracy: 0.9206 - val_loss: 4.9296 - val_accuracy: 0.6986
Epoch 3/20
45/45 [==============================] - 1s 19ms/step - loss: 0.1458 - accuracy: 0.9241 - val_loss: 5.0062 - val_accuracy: 0.6957
Epoch 4/20
45/45 [==============================] - 1s 17ms/step - loss: 0.1449 - accuracy: 0.9183 - val_loss: 5.7825 - val_accuracy: 0.7029
Epoch 5/20
45/45 [==============================] - 1s 16ms/step - loss: 0.1490 - accuracy: 0.9216 - val_loss: 4.8511 - val_accuracy: 0.6971
Epoch 6/20
45/45 [==============================] - 1s 16ms/step - loss: 0.1451 - accuracy: 0.9202 - val_loss: 5.1132 - val_accuracy: 0.6971
Epoch 7/20
45/45 [==============================] - 1s 16ms/step - loss: 0.1431 - accuracy: 0.9208 - val_loss: 5.0785 - val_accuracy: 0.6729
Epoch 8/20
45

Se llega a $92\%$ de precisión en los datos de entrenamiento y $70\%$ en los de validación.

In [26]:
index_word_dict = {i:w for w,i in word_index_dict.items()}
index_translated_word_dict = {i:w for w,i in translated_word_index_dict.items()}

In [27]:
def translate_sentence(input_seq):
    # Retorna los valores inciales de h/c que se generan con la oración input
    states_values = encoder_model.predict(input_seq)

    # Regresa el token de inicio e inicia el target_seq 
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = translated_word_index_dict['走']

    # Asigna el token del final a una variable y se para el for cuando se alcanza este valor
    eos = translated_word_index_dict['停']

    # Cada palabra del output se agrega a esta lista 
    output_sentence = []

    # Recorrer indefinidamente hasta que alcance el máximo largo de la oración
    for _ in range(max_output_length):
        # Retorna los estados predecidos 
        output_seq, h, c = decoder_model.predict([target_seq] + states_values)

        # Regresa el indice con mayor valor del arreglo del softmax 
        idx = np.argmax(output_seq[0, 0])
        if eos == idx: # si el indice es igual al del token de final se para el proceso
            break

        word = ''

        if idx > 0: # si el indice no es padding
            word = index_translated_word_dict[idx]
            output_sentence.append(word)


        target_seq[0, 0] = idx
        states_values = [h, c]

    # Retorna la oración output como un string.
    return ' '.join(output_sentence)

Tomamos 3 frases al azar para mostrar el desempeño. Teniendo $92\%$ de precisión nos hace esperar muy buenos resultados.*texto en cursiva*

In [39]:
count = 1
for _ in range(3):
    print(f'Translation {count}: ')
    i = np.random.choice(len(input_sentences))
    input_seq = encoder_input_sequences[i:i+1]
    translation = translate_sentence(input_seq)
    print('Input:', input_sentences[i])
    print('Response:', translation)
    print('')
    count += 1

Translation 1: 
1/1 [==============================] - 0s 19ms/step
Input: Move over
Response: veníos

Translation 2: 
1/1 [==============================] - 0s 33ms/step
Input: Hes a DJ
Response: él es dj

Translation 3: 
1/1 [==============================] - 0s 37ms/step
Input: Im happy
Response: estoy feliz



En efecto, el desempeño es bueno.

In [32]:
def prepare_sequence(sentence):
    sentence = re.sub(r'[^\w\s]','', sentence)
    sentence_array = sentence.rstrip().lower().split(' ')
    sentence_sequence = [word_index_dict[word] for word in sentence_array if word in word_index_dict.keys()]
    sentence_sequence = pad_sequences([sentence_sequence], maxlen=max_input_length)
    return sentence_sequence

Predicción con frases nuevas:

In [41]:
custom_sentences = [
    'This was really hard',
    'I did understood something',
    'Im tired of working',             
]

custom_sequences = []

for custom_sentence in custom_sentences:
    sequence = prepare_sequence(custom_sentence)
    custom_sequences.append(sequence)

for i, custom_sequence in enumerate(custom_sequences):
    translation = translate_sentence(custom_sequence)

    print('Input:', custom_sentences[i])
    print('Response:', translation)
    print('')

1/1 [==============================] - 0s 18ms/step
Input: This was really hard
Response: estoy duro

1/1 [==============================] - 0s 19ms/step
Input: I did understood something
Response: yo lo sé

1/1 [==============================] - 0s 17ms/step
Input: Im tired of working
Response: estoy lleno



# Conclusión. 

Se utilizan embeddings de inglés y español predefinidos para un encoder-decoder de frases a nivel palabras. 
Se obtiene buena precisión en los datos de entrenamiento ($90\%$) y no tan buenos en los datos de validación ($70\%$), lo que podría indicar que nuestra red se memoriza las frases del diccionario que se le pasa. 

Se hacen predicciones de frases que no necesariamente se encuentran en la base de datos y los resultados son malos, lo que parece confirmar que nuestra red se memorizó los datos de entrenamiento.